In [26]:
import numpy as np
import matplotlib.pyplot as plt

Ex = np.loadtxt("../main_files/Ex_t.dat")
f = np.loadtxt("../main_files/f.dat")
f_total = np.loadtxt("../main_files/f_total.dat")
j = np.loadtxt("../main_files/j.dat")
t = np.arange(Ex.shape[0])*20



In [ ]:
f_total[100]

In [ ]:
plt.plot(t,f_total)
plt.xlabel("time step")
plt.ylabel(r"$f_{total}  \lambda_D^3v_{th}^3$")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# ======================
# データ読み込み
# ======================
Ex = np.loadtxt("../main_files/Ex_t.dat")
f  = np.loadtxt("../main_files/f.dat")

# 平均除去（強く推奨）
# Ex = Ex - Ex.mean(axis=1, keepdims=True)

Nt, Nx = Ex.shape
x = np.arange(Nx)

# ======================
# アニメーション設定
# ======================
stride   = 49      # ← 何ステップごとに1コマ使うか
nframes  = 400    # ← 作りたいコマ数
interval = 50     # ms

max_frames = Nt // stride
nframes = min(nframes, max_frames)

frames = stride * np.arange(nframes)

# ======================
# 図の初期化
# ======================
fig, axes = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

line_f, = axes[0].plot(x, f[frames[0]], lw=2)
axes[0].set_ylabel("n(x)")
axes[0].set_title("Distribution function")

line_E, = axes[1].plot(x, Ex[frames[0]], lw=2, color='r')
axes[1].set_ylabel("Ex(x)")
axes[1].set_xlabel("x")

# 軸固定（超重要）
axes[0].set_ylim(f.min(), f.max())
axes[1].set_ylim(Ex.min(), Ex.max())

# ======================
# 更新関数
# ======================
def update(i):
    it = frames[i]
    line_f.set_ydata(f[it])
    line_E.set_ydata(Ex[it])
    fig.suptitle(f"t index = {it}")
    return line_f, line_E

# ======================
# アニメーション生成
# ======================
ani = FuncAnimation(
    fig, update,
    frames=nframes,
    interval=interval,
    blit=True
)

ani.save("f_and_E_stride5.gif", fps=20)



In [ ]:
import struct
import numpy as np
import cv2
import glob
import imageio
from concurrent.futures import ProcessPoolExecutor

# =============================
#   描画パラメータ (定数として定義)
# =============================
XMIN, XMAX = 0, 422.4
YMIN, YMAX = -16.5, 16.5
RES = 800

def world_to_img(x, y):
    ix = int((x - XMIN) / (XMAX - XMIN) * RES)
    iy = int((YMAX - y) / (YMAX - YMIN) * RES)
    return (ix, iy)

def process_frame(fn):
    """
    1つのファイルを読み込み、画像配列を生成して返す関数
    """
    try:
        with open(fn, "rb") as f:
            Nx = struct.unpack("q", f.read(8))[0]
            Ny = struct.unpack("q", f.read(8))[0]

            vx_all = np.zeros((Nx, Ny, 4))
            vy_all = np.zeros((Nx, Ny, 4))
            val_all = np.zeros((Nx, Ny))

            for i in range(Nx):
                for j in range(Ny):
                    vx_all[i,j] = struct.unpack("4d", f.read(32))
                    vy_all[i,j] = struct.unpack("4d", f.read(32))
                    val_all[i,j] = struct.unpack("d",  f.read(8))[0]

        # 画像化処理
        vmin, vmax = val_all.min(), val_all.max()
        val_norm = (val_all - vmin) / (vmax - vmin + 1e-12) * 255
        
        img = np.zeros((RES, RES, 3), dtype=np.uint8)
        for i in range(Nx):
            for j in range(Ny):
                pts = np.array([
                    world_to_img(vx_all[i,j,k], vy_all[i,j,k])
                    for k in range(4)
                ], np.int32)
                color = int(val_norm[i,j])
                cv2.fillPoly(img, [pts], (0, 0, color))
        
        print(f"Finished: {fn}")
        return img
    except Exception as e:
        print(f"Error in {fn}: {e}")
        return None

# =============================
#   メイン処理
# =============================
if __name__ == "__main__":
    # ファイルリストの取得とフィルタリング
    all_files = sorted(glob.glob("../output/two_stream/*.bin"), 
                       key=lambda s: int(s.split('/')[-1].split('.')[0]))
    
    # 条件: i%4 != 0 かつ i < 5000 のファイルのみ抽出
    target_files = [fn for i, fn in enumerate(all_files) if i % 4 == 0 and i < 5000]

    print(f"Total frames to process: {len(target_files)}")

    # プロセスプールを使用して並列実行
    # executor.map は元のリストの順序を維持して結果を返します
    with ProcessPoolExecutor() as executor:
        frames = list(executor.map(process_frame, target_files))

    # None（エラー）を除外
    frames = [f for f in frames if f is not None]

    # GIF 保存
    gif_path = "../output/two_stream_instability.gif"
    fps = 15
    imageio.mimsave(gif_path, frames, fps=fps)

    print(f"GIF saved to {gif_path}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ======================
# 物理パラメータ
# ======================
c = 3e8
omega_pe = 564102.5
v_th = 0.003 * c
debye_length = v_th / omega_pe

dx = 0.99 * debye_length
dt = 0.01 / omega_pe

# ======================
# データ読み込み
# ======================
Ex = np.loadtxt("../main_files/Ex_t.dat")  # (Nt, Nx)
Nt, Nx = Ex.shape

# ======================
# 必須①：時間平均除去（ω=0対策）
# ======================
Ex = Ex - Ex.mean(axis=0, keepdims=True)

# ======================
# FFT
# ======================
E_k = np.fft.fft(Ex, axis=1)
k = 2*np.pi*np.fft.fftfreq(Nx, d=dx)

E_wk = np.fft.fft(E_k, axis=0)
omega = 2*np.pi*np.fft.fftfreq(Nt, d=dt)

power = np.abs(E_wk)**2

# ======================
# 必須②：ω>0 のみ
# ======================
om_mask = omega > 0
k_mask = k > 0

omega = omega[om_mask]
k = k[k_mask]
power = power[np.ix_(om_mask, k_mask)]

# ======================
# kごと正規化
# ======================
#power /= power.max(axis=0, keepdims=True)

# ======================
# log 表示
# ======================
log_power = np.log10(power + 1e-12)

# ======================
# 軸の選択
# ======================
use_normalized_axis = True

omega_analysis = np.sqrt(1. + 3.*((k*debye_length)**2))*omega_pe

if use_normalized_axis:
    k_plot = k * debye_length
    omega_plot = omega / omega_pe
    omega_analysis_plot = omega_analysis / omega_pe
    xlabel = r"$k\lambda_D$"
    ylabel = r"$\omega/\omega_{pe}$"
    #ylim = (0, 2.0)
else:
    k_plot = k
    omega_plot = omega
    omega_analysis_plot = omega_analysis
    xlabel = r"$k\,[\mathrm{rad/m}]$"
    ylabel = r"$\omega\,[\mathrm{rad/s}]$"
    ylim = (0, omega.max())

# ======================
# 描画
# ======================
plt.figure(figsize=(8,6))
plt.pcolormesh(k_plot, omega_plot, log_power,
               shading="auto", cmap="inferno")
plt.colorbar(label=r"$\log_{10}$ power")
plt.plot(k_plot, omega_analysis_plot, 
         color="black", lw = 0.5,
         linestyle="--", label="Theory (Bohm-Gross)")
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.xlim((0.,1.5))
plt.ylim((0.,2.))
plt.tight_layout()
plt.show()
